## Домашнее задание 6
4. Проэкспериментриуйте с различными моделями recbole и выберите лучшую. (4 балла)

In [1]:
!pip install recbole ray >> None

In [2]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
interactions_df = pd.read_csv('/content/drive/MyDrive/data/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/data/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/data/items_processed_kion.csv')

In [6]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [7]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [8]:
!mkdir recbox_data

In [9]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [10]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [11]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [12]:
dataset = create_dataset(config)
logger.info(dataset)

In [13]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [14]:
import time

потестим несколько моделей

In [17]:
%%time
model_list = ['ADMMSLIM'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running ADMMSLIM...


Evaluate   : 100%|███████████████████| 13354/13354 [07:47<00:00, 28.57it/s, GPU RAM: 0.00 G/14.76 G]


It took 11.15 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0982), ('mrr@10', 0.1872), ('ndcg@10', 0.0935), ('hit@10', 0.4019), ('precision@10', 0.0542)])}
CPU times: user 10min 8s, sys: 3min 41s, total: 13min 50s
Wall time: 11min 8s


In [25]:
%%time
model_list = ['BPR'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running BPR...


Evaluate   : 100%|███████████████████| 13354/13354 [07:46<00:00, 28.61it/s, GPU RAM: 0.05 G/14.76 G]


It took 11.67 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0824), ('mrr@10', 0.1716), ('ndcg@10', 0.0819), ('hit@10', 0.3479), ('precision@10', 0.0457)])}
CPU times: user 9min 22s, sys: 59.2 s, total: 10min 21s
Wall time: 11min 40s


In [15]:
%%time
model_list = ['CDAE'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running CDAE...


Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [04:04<00:00, 54.61it/s, GPU RAM: 0.37 G/14.76 G]


It took 5.06 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.1428), ('ndcg@10', 0.0657), ('hit@10', 0.2724), ('precision@10', 0.0344)])}
CPU times: user 4min 24s, sys: 16.4 s, total: 4min 40s
Wall time: 5min 3s


In [25]:
%%time
model_list = ['ConvNCF'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running ConvNCF...


Evaluate   : 100%|███████████████████| 13354/13354 [10:17<00:00, 21.63it/s, GPU RAM: 0.70 G/14.76 G]


It took 19.92 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0618), ('mrr@10', 0.1353), ('ndcg@10', 0.063), ('hit@10', 0.2675), ('precision@10', 0.0334)])}
CPU times: user 17min 6s, sys: 1min 24s, total: 18min 31s
Wall time: 19min 55s


модели с урока

In [ ]:
%%time
model_list = ['MultiVAE', 'MultiDAE', 'MacridVAE', 
              "NeuMF", "RecVAE", 'RepeatNet'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running MultiVAE...


Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [04:49<00:00, 46.11it/s, GPU RAM: 0.36 G/14.76 G]


It took 6.13 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0834), ('mrr@10', 0.1671), ('ndcg@10', 0.0816), ('hit@10', 0.3466), ('precision@10', 0.0462)])}
running MultiDAE...


Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [06:15<00:00, 35.60it/s, GPU RAM: 0.38 G/14.76 G]


It took 7.20 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0837), ('mrr@10', 0.1657), ('ndcg@10', 0.0814), ('hit@10', 0.3466), ('precision@10', 0.0463)])}
running MacridVAE...


...

It took 16.23 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0687), ('mrr@10', 0.1187), ('ndcg@10', 0.0608), ('hit@10', 0.3019), ('precision@10', 0.038)])}
running RecVAE...


Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [08:55<00:00, 24.94it/s, GPU RAM: 0.93 G/14.76 G]


It took 10.43 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0846), ('mrr@10', 0.1661), ('ndcg@10', 0.0818), ('hit@10', 0.3523), ('precision@10', 0.0469)])}
running RepeatNet...


Train     2:  94%|█████████████████████▋ | 701/743 [15:23<00:55,  1.32s/it, GPU RAM: 4.00 G/14.76 G]


KeyboardInterrupt: ignored

лучшая модель - ADMMSLIM - Sparse Recommendations for Many Users